In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import collections
import nest_asyncio
%reload_ext tensorboard
nest_asyncio.apply()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

In [24]:
emnist_train.client_ids[3382]

'f4099_10'

In [3]:
def clientDataFromTFF(tffDataset):
    NUM_EPOCHS = 10
    BATCH_SIZE = 20
    SHUFFLE_BUFFER = 100
    PREFETCH_BUFFER= 10

    def preprocess(dataset):
        def batch_format_fn(element):
            """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
            return collections.OrderedDict(
                x=tf.reshape(element['pixels'], [-1, 784]),
                y=tf.reshape(element['label'], [-1, 1]))

        return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
          BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

    return tfds.as_numpy(preprocess(tffDataset.create_tf_dataset_for_client(tffDataset.client_ids[2])))

In [4]:
clientData = clientDataFromTFF(emnist_train)

In [5]:
clientData

In [5]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(784,)),
            tf.keras.layers.Dense(10, kernel_initializer='zeros'),
            tf.keras.layers.Softmax()])

In [6]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.SGD (learning_rate=0.02),
              metrics=['accuracy'])

In [7]:
currentData = next(iter(clientData))

In [12]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


RuntimeError: Physical devices cannot be modified after being initialized

In [8]:
model.fit(currentData['x'],currentData['y'],epochs=10,verbose=1,shuffle=True)

Epoch 1/10


InternalError:  Blas GEMM launch failed : a.shape=(20, 784), b.shape=(784, 10), m=20, n=10, k=784
	 [[node sequential/dense/MatMul (defined at <ipython-input-8-d9fa882804ce>:1) ]] [Op:__inference_train_function_428]

Function call stack:
train_function


In [10]:
currentData = next(iter(clientData))

In [20]:
predictions = model.predict(currentData['x'])

In [26]:
type([predictions])
print(len(predictions))

20


In [22]:
predictions[0]

array([0.25055683, 0.00128346, 0.129285  , 0.14008793, 0.15099151,
       0.00335698, 0.00335698, 0.13652723, 0.04920679, 0.13534726],
      dtype=float32)

In [23]:
k=predictions[0]

In [24]:
type(k)

numpy.ndarray

In [25]:
k[0]

0.25055683

In [33]:
# Find Score Entropy and Data Quality
entropy = []*len(predictions)
for i in range (0,len(predictions)):
    sampleClassProb = predictions[i]
    score=0
    for value in sampleClassProb:
        print('Value:',value)
        score += value*np.log2(value)
    score=score*(-1)
    print('Score:',score)
    entropy.append(score)
print('Entropy:',entropy)

Value: 0.25055683
Value: 0.001283455
Value: 0.129285
Value: 0.14008793
Value: 0.15099151
Value: 0.0033569844
Value: 0.0033569844
Value: 0.13652723
Value: 0.049206786
Value: 0.13534726
Score: 2.754964869469404
Value: 0.22862925
Value: 0.001293132
Value: 0.14746533
Value: 0.13942814
Value: 0.1576642
Value: 0.0035103299
Value: 0.0035103299
Value: 0.13748445
Value: 0.052108496
Value: 0.12890637
Score: 2.776790814474225
Value: 0.22599098
Value: 0.0014675119
Value: 0.13126197
Value: 0.14510415
Value: 0.14611869
Value: 0.0031859684
Value: 0.0031859684
Value: 0.15129706
Value: 0.048941728
Value: 0.14344598
Score: 2.772732828743756
Value: 0.2152273
Value: 0.0014248865
Value: 0.13386586
Value: 0.14411724
Value: 0.14425848
Value: 0.0032456443
Value: 0.0032456443
Value: 0.14592998
Value: 0.05140721
Value: 0.15727769
Score: 2.7832046914845705
Value: 0.20326476
Value: 0.0013381235
Value: 0.12634009
Value: 0.14405367
Value: 0.14907491
Value: 0.0034820575
Value: 0.0034820575
Value: 0.14566328
Value: 0

In [34]:
max(entropy)

2.801827789284289

In [54]:
k=[4,4,7,9,1,0,2,3]
p=[0,1,2,3,4,5,6,7]


In [55]:
m, n = ( list(t) for t in zip(*sorted(zip(k,p))))

In [56]:
print(m,n)

[0, 1, 2, 3, 4, 4, 7, 9] [5, 4, 6, 7, 0, 1, 2, 3]


In [58]:
maxM = m[-1:][0]
maxM

9

In [59]:
newM = [i for i in m if i >= maxM*.6]
newM

[7, 9]

In [61]:
len(newM)

2

In [62]:
newN = n[-len(newM):]

In [63]:
newN

[2, 3]

In [53]:
m1=m[-2:]
n1=n[-1:][0]
m1,n1

([7, 9], 3)

In [46]:
2.45

2.45

In [60]:
9*.6

5.3999999999999995